In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

list=[['2019-02-23','2019-02-20',          2],
      ['2019-03-20','2019-02-20',          7],
      ['2019-03-21', '2019-02-21',         12],
      ['2019-03-22', '2019-02-22',         27],
      ['2019-03-23', '2019-02-23',         91]]

df=spark.createDataFrame(list,['AsofDate','oneMonthAgo','value'])

df.show()

In [3]:
list=[[10      ,       'Blue'],
      [5        ,      'Green'],
      [21        ,     'Red']]

df=spark.createDataFrame(list,['Count','Value'])

df.show()

+-----+-----+
Count|Value|
+-----+-----+
 10| Blue|
 5|Green|
 21| Red|
+-----+-----+

In [4]:
df.withColumn("map", F.create_map("Value","Count")).select("map").toJSON().collect()

Out[43]: ['{"map":{"Blue":10}}', '{"map":{"Green":5}}', '{"map":{"Red":21}}']

In [5]:
print(a)

MapPartitionsRDD[1017] at toJavaRDD at NativeMethodAccessorImpl.java:0

In [6]:
cDf = spark.createDataFrame([(None, None), (1, None), (None, 2),(1,2)], ("a", "b"))
cDf.show()

cDf.select(coalesce(cDf["a"], cDf["b"])).show()

+----+----+
 a| b|
+----+----+
null|null|
 1|null|
null| 2|
 1| 2|
+----+----+

+--------------+
coalesce(a, b)|
+--------------+
 null|
 1|
 2|
 1|
+--------------+

In [7]:
list=[[ 'Copper',   '2019-01-09',        2.6945,                 2.6838],
      ['Copper',   '2019-01-23',        2.6838,                 2.6838],
      ['Zinc',  '2019-01-23',        1.1829,                 1.1829],
      ['Zinc',   '2019-06-26',        1.1918,                 1.1918],
      ['Aluminum',   '2019-01-02',        0.8363,                 0.8342],
      ['Aluminum',   '2019-01-09',        0.8342,                 0.8342],
      ['Aluminum',   '2019-01-23',        0.8555,                 0.8342],
      ['Aluminum',   '2019-04-03',        0.8461,                 0.8461],
      ['Aluminum',   '2019-05-03',        0.7461,                 0.7461],
     ['Aluminum',   '2019-05-09',        0.9123,                 0.7461],
     ['Aluminum',   '2019-06-15',        0.6515,                 0.6515]]

df=spark.createDataFrame(list,['material','purchase_date','mkt_prc_usd_lb','min_mkt_prc_over_1month'])


df.show()

+--------+-------------+--------------+-----------------------+
material|purchase_date|mkt_prc_usd_lb|min_mkt_prc_over_1month|
+--------+-------------+--------------+-----------------------+
 Copper| 2019-01-09| 2.6945| 2.6838|
 Copper| 2019-01-23| 2.6838| 2.6838|
 Zinc| 2019-01-23| 1.1829| 1.1829|
 Zinc| 2019-06-26| 1.1918| 1.1918|
Aluminum| 2019-01-02| 0.8363| 0.8342|
Aluminum| 2019-01-09| 0.8342| 0.8342|
Aluminum| 2019-01-23| 0.8555| 0.8342|
Aluminum| 2019-04-03| 0.8461| 0.8461|
Aluminum| 2019-05-03| 0.7461| 0.7461|
Aluminum| 2019-05-09| 0.9123| 0.7461|
Aluminum| 2019-06-15| 0.6515| 0.6515|
+--------+-------------+--------------+-----------------------+

In [8]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

days= lambda i: i * 86400
w2 = (Window()
           .partitionBy("material")
           .orderBy(col("purchase_date").cast("timestamp").cast("long"))
           .rangeBetween(-days(15), days(15)))


df.withColumn("first",\
              expr("""IF(mkt_prc_usd_lb=min_mkt_prc_over_1month,purchase_date,null)"""))\
  .withColumn("second", first("first", True).over(w2))\
  .withColumn("date_of_min_price", coalesce("first","second"))\
  .show()

#+--------+-------------+--------------+-----------------------+-----------------+
#|material|purchase_date|mkt_prc_usd_lb|min_mkt_prc_over_1month|date_of_min_price|
#+--------+-------------+--------------+-----------------------+-----------------+
#|  Copper|   2019-01-09|        2.6945|                 2.6838|       2019-01-23|
#|  Copper|   2019-01-23|        2.6838|                 2.6838|       2019-01-23|
#|    Zinc|   2019-01-23|        1.1829|                 1.1829|       2019-01-23|
#|    Zinc|   2019-06-26|        1.1918|                 1.1918|       2019-06-26|
#|Aluminum|   2019-01-02|        0.8363|                 0.8342|       2019-01-09|
#|Aluminum|   2019-01-09|        0.8342|                 0.8342|       2019-01-09|
#|Aluminum|   2019-01-23|        0.8555|                 0.8342|       2019-01-09|
#|Aluminum|   2019-04-03|        0.8461|                 0.8461|       2019-04-03|
#+--------+-------------+--------------+-----------------------+-----------------+

+--------+-------------+--------------+-----------------------+----------+----------+-----------------+
material|purchase_date|mkt_prc_usd_lb|min_mkt_prc_over_1month| first| second|date_of_min_price|
+--------+-------------+--------------+-----------------------+----------+----------+-----------------+
 Copper| 2019-01-09| 2.6945| 2.6838| null|2019-01-23| 2019-01-23|
 Copper| 2019-01-23| 2.6838| 2.6838|2019-01-23|2019-01-23| 2019-01-23|
 Zinc| 2019-01-23| 1.1829| 1.1829|2019-01-23|2019-01-23| 2019-01-23|
 Zinc| 2019-06-26| 1.1918| 1.1918|2019-06-26|2019-06-26| 2019-06-26|
Aluminum| 2019-01-02| 0.8363| 0.8342| null|2019-01-09| 2019-01-09|
Aluminum| 2019-01-09| 0.8342| 0.8342|2019-01-09|2019-01-09| 2019-01-09|
Aluminum| 2019-01-23| 0.8555| 0.8342| null|2019-01-09| 2019-01-09|
Aluminum| 2019-04-03| 0.8461| 0.8461|2019-04-03|2019-04-03| 2019-04-03|
Aluminum| 2019-05-03| 0.7461| 0.7461|2019-05-03|2019-05-03| 2019-05-03|
Aluminum| 2019-05-09| 0.9123| 0.7461| null|2019-05-03| 2019-05-03|
Aluminum| 2019-06-15| 0.6515| 0.6515|2019-06-15|2019-06-15| 2019-06-15|
+--------+-------------+--------------+-----------------------+----------+----------+-----------------+

In [9]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

days= lambda i: i * 86400
w2 = (Window()
           .partitionBy("material")
           .orderBy(col("purchase_date").cast("timestamp").cast("long"))
           .rangeBetween(-days(15), days(15)))


df.withColumn("first",\
              expr("""IF(mkt_prc_usd_lb=min_mkt_prc_over_1month,purchase_date,null)"""))\
  .withColumn("date_of_min_price", first("first", True).over(w2)).drop("first")\
  .show()

#+--------+-------------+--------------+-----------------------+-----------------+
#|material|purchase_date|mkt_prc_usd_lb|min_mkt_prc_over_1month|date_of_min_price|
#+--------+-------------+--------------+-----------------------+-----------------+
#|  Copper|   2019-01-09|        2.6945|                 2.6838|       2019-01-23|
#|  Copper|   2019-01-23|        2.6838|                 2.6838|       2019-01-23|
#|    Zinc|   2019-01-23|        1.1829|                 1.1829|       2019-01-23|
#|    Zinc|   2019-06-26|        1.1918|                 1.1918|       2019-06-26|
#|Aluminum|   2019-01-02|        0.8363|                 0.8342|       2019-01-09|
#|Aluminum|   2019-01-09|        0.8342|                 0.8342|       2019-01-09|
#|Aluminum|   2019-01-23|        0.8555|                 0.8342|       2019-01-09|
#|Aluminum|   2019-04-03|        0.8461|                 0.8461|       2019-04-03|
#+--------+-------------+--------------+-----------------------+-----------------+

+--------+-------------+--------------+-----------------------+-----------------+
material|purchase_date|mkt_prc_usd_lb|min_mkt_prc_over_1month|date_of_min_price|
+--------+-------------+--------------+-----------------------+-----------------+
 Copper| 2019-01-09| 2.6945| 2.6838| 2019-01-23|
 Copper| 2019-01-23| 2.6838| 2.6838| 2019-01-23|
 Zinc| 2019-01-23| 1.1829| 1.1829| 2019-01-23|
 Zinc| 2019-06-26| 1.1918| 1.1918| 2019-06-26|
Aluminum| 2019-01-02| 0.8363| 0.8342| 2019-01-09|
Aluminum| 2019-01-09| 0.8342| 0.8342| 2019-01-09|
Aluminum| 2019-01-23| 0.8555| 0.8342| 2019-01-09|
Aluminum| 2019-04-03| 0.8461| 0.8461| 2019-04-03|
Aluminum| 2019-05-03| 0.7461| 0.7461| 2019-05-03|
Aluminum| 2019-05-09| 0.9123| 0.7461| 2019-05-03|
Aluminum| 2019-06-15| 0.6515| 0.6515| 2019-06-15|
+--------+-------------+--------------+-----------------------+-----------------+

In [10]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

days= lambda i: i * 86400
w2 = (Window()
           .partitionBy("material")
           .orderBy(col("purchase_date").cast("timestamp").cast("long"))
           .rangeBetween(-days(15), days(15)))


df.withColumn("first",\
              expr("""IF(mkt_prc_usd_lb=min_mkt_prc_over_1month,purchase_date,null)"""))\
  .withColumn("date_of_min_price", first("first", True).over(w2)).drop("first")\
  .show()

+--------+-------------+--------------+-----------------------+-----------------+
material|purchase_date|mkt_prc_usd_lb|min_mkt_prc_over_1month|date_of_min_price|
+--------+-------------+--------------+-----------------------+-----------------+
 Copper| 2019-01-09| 2.6945| 2.6838| 2019-01-23|
 Copper| 2019-01-23| 2.6838| 2.6838| 2019-01-23|
 Zinc| 2019-01-23| 1.1829| 1.1829| 2019-01-23|
 Zinc| 2019-06-26| 1.1918| 1.1918| 2019-06-26|
Aluminum| 2019-01-02| 0.8363| 0.8342| 2019-01-09|
Aluminum| 2019-01-09| 0.8342| 0.8342| 2019-01-09|
Aluminum| 2019-01-23| 0.8555| 0.8342| 2019-01-09|
Aluminum| 2019-04-03| 0.8461| 0.8461| 2019-04-03|
Aluminum| 2019-05-03| 0.7461| 0.7461| 2019-05-03|
Aluminum| 2019-05-09| 0.9123| 0.7461| 2019-05-03|
Aluminum| 2019-06-15| 0.6515| 0.6515| 2019-06-15|
+--------+-------------+--------------+-----------------------+-----------------+

In [11]:
days= lambda i: i * 86400
w2 = (Window()
           .partitionBy("material")
           .orderBy(col("purchase_date").cast("timestamp").cast("long"))
           .rangeBetween(-days(15), days(15)))


df.withColumn("date_of_min_price",\
              F.when(F.col("mkt_prc_usd_lb")==F.col("min_mkt_prc_over_1month"),F.col("purchase_date"))\
               .otherwise(F.first("date_of_min_price", True).over(w2))).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o8989.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve '`date_of_min_price`' given input columns: [material, purchase_date, mkt_prc_usd_lb, min_mkt_prc_over_1month];;
'Project [material#7592, purchase_date#7593, mkt_prc_usd_lb#7594, min_mkt_prc_over_1month#7595, CASE WHEN (mkt_prc_usd_lb#7594 = min_mkt_prc_over_1month#7595) THEN purchase_date#7593 ELSE first('date_of_min_price, true) windowspecdefinition(material#7592, cast(cast(purchase_date#7593 as timestamp) as bigint) ASC NULLS FIRST, specifiedwindowframe(RangeFrame, -1296000, 1296000)) END AS date_of_min_price#9583]
+- LogicalRDD [material#7592, purchase_date#7593, mkt_prc_usd_lb#7594, min_mkt_prc_over_1month#7595], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:120)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:361)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.

In [12]:
from pyspark.sql.functions import *

days= lambda i: i * 86400
w2 = (Window()
           .partitionBy("material")
           .orderBy(col("purchase_date").cast("timestamp").cast("long"))
           .rangeBetween(-days(15), days(15)))

df.withColumn('d', F.expr("min(struct(mkt_prc_usd_lb as min_mkt_prc_over_1month, purchase_date as date_of_min_price))").over(w2)).select("*", "d.*").drop('d').show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4162875704520561> in <module> 
 6 . orderBy ( col ( "purchase_date" ) . cast ( "timestamp" ) . cast ( "long" ) ) 
 7 .rangeBetween(-days(15), days(15)))
 ----> 8 df . withColumn ( 'd' , F . expr ( "min(struct(mkt_prc_usd_lb as min_mkt_prc_over_1month, purchase_date as date_of_min_price))" ) . over ( w2 ) ) . select ( "*" , "d.*" ) . drop ( 'd' ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 382 """
 383 if isinstance ( truncate , bool ) and truncate : 
 --> 384 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 385 else : 
 386 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o9436.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 426.0 failed 1 times, most recent failure: Lost task 1.0 in stage 426.0 (TID 4803, localhost, executor driver): java.lang.ClassCastException: org.apache.spark.sql.catalyst.expressions.UnsafeRow cannot be cast to java.lang.Comparable
	at java.util.PriorityQueue.siftUpComparable(PriorityQueue.java:653)
	at java.util.PriorityQueue.siftUp(PriorityQueue.java:648)
	at java.util.PriorityQueue.offer(PriorityQueue.java:345)
	at java.util.PriorityQueue.add(PriorityQueue.java:322)
	at com.databricks.sql.catalyst.expressions.aggregation.removable.MinMaxRemovableLike.update(MinMax.scala:50)
	at com.databricks.sql.catalyst.expressions.aggregation.removable.MinMaxRemovableLike.update(MinMax.scala:35)
	at org.apache.spark.sql.catalyst.expressions.aggregate.TypedImperativeAggregate.update(interfaces.scala:692)
	at com.databricks.sql.catalyst.expressions.aggregation.removable.RemovableAggregateProcessor.add(RemovableAggregateProcessor.scala:161)
	at org.apache.spark.sql.execution.window.RemovableSlidingWindowFunctionFrame.write(WindowFunctionFrame.scala:483)
	at org.apache.spark.sql.execution.window.WindowExec$$anonfun$14$$anon$1.next(WindowExec.scala:516)
	at org.apache.spark.sql.execution.window.WindowExec$$anonfun$14$$anon$1.next(WindowExec.scala:413)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:62)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:159)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:158)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:140)
	at org.apache.spark.scheduler.Task.run(Task.scala:113)
	at org.apache.spark.executor.Executor$TaskRunner$$anonfun$13.apply(Executor.scala:537)
	at org.apache.spark.util.Utils$.try

In [13]:
list=[['0023' ,    'g'],
      ['0025' ,    'h'],
      ['0026' ,    'x'],
      ['0031' ,    'y'],
      ['0034' ,    'z']]

df=spark.createDataFrame(list,['time','data'])

df.show()

+----+----+
time|data|
+----+----+
0023| g|
0025| h|
0026| x|
0031| y|
0034| z|
+----+----+

In [14]:
df.show() #sample dataframe
#+----+----+
#|time|data|
#+----+----+
#|0023|   g|
#|0025|   h|
#|0026|   x|
#|0031|   y|
#|0034|   z|
#+----+----+


from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().orderBy("time")
df.withColumn("lag1", F.lag("time").over(w)).withColumn("lag2", F.lag("data").over(w))\
  .withColumn("collect", F.sort_array(F.collect_list(F.array("lag1","lag2")).over(w),False)).drop("lag1","lag2")\
  .withColumn("collect", F.expr("""filter(collect,(x,i)-> i<=1)"""))\
  .withColumn("collect", F.when(F.size("collect")<2, F.flatten(F.array_repeat("collect",2)))\
                          .otherwise(F.col("collect")))\
  .select("time","data",*[(F.col("collect")[x][y]).alias("{}".format(z))\
                          for x,y,z in zip([0,0,1,1],[0,1,0,1],['time1','data1','time2','data2'])])\
  .show(truncate=False)


#+----+----+-----+-----+-----+-----+
#|time|data|time1|data1|time2|data2|
#+----+----+-----+-----+-----+-----+
#|0023|g   |null |null |null |null |
#|0025|h   |0023 |g    |null |null |
#|0026|x   |0025 |h    |0023 |g    |
#|0031|y   |0026 |x    |0025 |h    |
#|0034|z   |0031 |y    |0026 |x    |
#+----+----+-----+-----+-----+-----+

#+----+----+-----+-----+-----+-----+
#|time|data|time1|data1|time2|data2|
#+----+----+-----+-----+-----+-----+
#|0023|g   |null |null |null |null |
#|0025|h   |0023 |g    |null |null |
#|0026|x   |0025 |h    |0023 |g    |
#|0031|y   |0026 |x    |0025 |h    |
#|0034|z   |0031 |y    |0026 |x    |
#+----+----+-----+-----+-----+-----+

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3228451159491666> in <module> () 
 15 
 16 w = Window ( ) . orderBy ( "time" ) 
 ---> 17 df . withColumn ( "lag1" , F . lag ( "time" ) . over ( w ) ) . withColumn ( "lag2" , F . lag ( "data" ) . over ( w ) ) . withColumn ( "collect" , F . sort_array ( F . collect_list ( F . array ( "lag1" , "lag2" ) ) . over ( w ) , False ) ) . drop ( "lag1" , "lag2" ) . withColumn ( "collect" , F . expr ( """filter(collect,(x,i)-> i<=1)""" ) ) . withColumn ( "collect" , F . when ( F . size ( "collect" ) < 2 , F . flatten ( F . array_repeat ( "collect" , 2 ) ) ) . otherwise ( F . col ( "collect" ) ) ) . select ( "time" , "data" , * [ ( F . col ( "collect" ) [ x ] [ y ] ) . alias ( "{}" . format ( z ) ) for x , y , z in zip ( [ 0 , 0 , 1 , 1 ] , [ 0 , 1 , 0 , 1 ] , [ 'time1' , 'data1' , 'time2' , 'data2' ] ) ] ) . show ( truncate = False ) 
 18 
 19 

 /databricks/spark/python/pyspark/sql/dataframe.py in withColumn (self, colName, col) 
 2012 """
 2013 assert isinstance ( col , Column ) , "col should be Column" 
 -> 2014 return DataFrame ( self . _jdf . withColumn ( colName , col . _jc ) , self . sql_ctx ) 
 2015 
 2016 @ ignore_unicode_prefix 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : "The number of lambda function arguments '2' does not match the number of arguments expected by the higher order function '1'.; line 1 pos 15"

In [15]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().orderBy("time")
df.withColumn("lag1", F.lag("time").over(w)).withColumn("lag2", F.lag("data").over(w))\
  .withColumn("collect", F.sort_array(F.collect_list(F.array("lag1","lag2")).over(w),False)).drop("lag1","lag2")\
  .withColumn("collect", F.when(F.size("collect")>1,F.array(F.element_at("collect",1),F.element_at("collect",2)))\
                          .otherwise(F.col("collect")))\
  .withColumn("collect", F.when(F.size("collect")<2, F.flatten(F.array_repeat("collect",2)))\
                          .otherwise(F.col("collect")))\
  .select("time","data",*[(F.col("collect")[x][y]).alias("{}".format(z))\
                          for x,y,z in zip([0,0,1,1],[0,1,0,1],['time1','data1','time2','data2'])])\
  .show(truncate=False)


+----+----+-----+-----+-----+-----+
time|data|time1|data1|time2|data2|
+----+----+-----+-----+-----+-----+
0023|g |null |null |null |null |
0025|h |0023 |g |null |null |
0026|x |0025 |h |0023 |g |
0031|y |0026 |x |0025 |h |
0034|z |0031 |y |0026 |x |
+----+----+-----+-----+-----+-----+

In [16]:

from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().orderBy("time")
df.withColumn("lag1", F.lag("time").over(w)).withColumn("lag2", F.lag("data").over(w))\
  .withColumn("collect", F.sort_array(F.collect_list(F.array("lag1","lag2")).over(w),False)).drop("lag1","lag2")\
  .withColumn("collect", F.expr("""filter(collect,(x,i)-> i<=1)"""))\
  .show(truncate=False)


--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-4034056067542266> in <module> () 
 4 
 5 w = Window ( ) . orderBy ( "time" ) 
 ----> 6 df . withColumn ( "lag1" , F . lag ( "time" ) . over ( w ) ) . withColumn ( "lag2" , F . lag ( "data" ) . over ( w ) ) . withColumn ( "collect" , F . sort_array ( F . collect_list ( F . array ( "lag1" , "lag2" ) ) . over ( w ) , False ) ) . drop ( "lag1" , "lag2" ) . withColumn ( "collect" , F . expr ( """filter(collect,(x,i)-> i<=1)""" ) ) . show ( truncate = False ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in withColumn (self, colName, col) 
 2012 """
 2013 assert isinstance ( col , Column ) , "col should be Column" 
 -> 2014 return DataFrame ( self . _jdf . withColumn ( colName , col . _jc ) , self . sql_ctx ) 
 2015 
 2016 @ ignore_unicode_prefix 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : "The number of lambda function arguments '2' does not match the number of arguments expected by the higher order function '1'.; line 1 pos 15"

In [17]:
w=Window().orderBy("time").rowsBetween(-2,Window.currentRow)
df.withColumn("collect", F.sort_array(F.collect_list(F.array("time","data")).over(w),False)).show(truncate=False)

In [18]:
df.withColumn("collect", F.sort_array(F.collect_list(F.array("time","data")).over(w),False)).show(truncate=False)

+----+----+-------------------------------------------------------+
time|data|collect |
+----+----+-------------------------------------------------------+
0023|g |[[0023, g]] |
0025|h |[[0025, h], [0023, g]] |
0026|x |[[0026, x], [0025, h], [0023, g]] |
0031|y |[[0031, y], [0026, x], [0025, h], [0023, g]] |
0034|z |[[0034, z], [0031, y], [0026, x], [0025, h], [0023, g]]|
+----+----+-------------------------------------------------------+

In [19]:
cols=[1,2]

w=Window().orderBy("time")
w1=Window().partitionBy()
df.withColumn("lag1", F.lag("time").over(w))\
  .withColumn("lag2", F.lag("data").over(w))\
  .withColumn("collect", F.sort_array(F.collect_list(F.array("lag1","lag2")).over(w),False))\
  .withColumn("max", F.max(F.size("collect")).over(w1))\
  .withColumn("size", F.col("max")-F.size("collect"))\
  .withColumn("array_repeat", F.expr("""array_repeat(array(null,null),size-1)"""))\
  .withColumn("flatten", F.when(F.col("max")!=F.size("collect"),\
                                F.flatten(F.array("collect","array_repeat")))\
                          .otherwise(F.expr("""filter(collect,(x,i)-> i!=max-1)""")))\
  .select("time","data","flatten").show(truncate=False)

+----+----+--------------------------------------------+
time|data|flatten |
+----+----+--------------------------------------------+
0023|g |[[,], [,], [,], [,]] |
0025|h |[[0023, g], [,], [,], [,]] |
0026|x |[[0025, h], [0023, g], [,], [,]] |
0031|y |[[0026, x], [0025, h], [0023, g], [,]] |
0034|z |[[0031, y], [0026, x], [0025, h], [0023, g]]|
+----+----+--------------------------------------------+

In [20]:
df.show() #sample dataframe
#+----+----+
#|time|data|
#+----+----+
#|0023|   g|
#|0025|   h|
#|0026|   x|
#|0031|   y|
#|0034|   z|
#+----+----+

from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().orderBy("time").rowsBetween(-2,Window.currentRow)

df.withColumn("collect", F.sort_array(F.collect_list(F.array("time","data")).over(w),False))
  .withColumn("collect", F.expr("""filter(collect,(x,i)-> i<=1)"""))\
  .withColumn("collect", F.when(F.size("collect")<2, F.flatten(F.array_repeat("collect",2)))\
                          .otherwise(F.col("collect")))\
  .select("time","data",(F.col("collect")[0][0]).alias("time1"),\
                         (F.col("collect")[0][1]).alias("data1"),\
                         (F.col("collect")[1][0]).alias("time2"),\
                         (F.col("collect")[1][1]).alias("data2"))\
  .show(truncate=False)


*[F.col("collet")[x][y]).alias("{}".format(z)) for x,y,z in zip([0,0,1,1],[0,1,0,1],['time1','data1','time2','data2'])]

+----+----+-----+-----+-----+-----+
time|data|time1|data1|time2|data2|
+----+----+-----+-----+-----+-----+
0023|g |null |null |null |null |
0025|h |0023 |g |null |null |
0026|x |0025 |h |0023 |g |
0031|y |0026 |x |0025 |h |
0034|z |0031 |y |0026 |x |
+----+----+-----+-----+-----+-----+

In [21]:
df.show() #sample dataframe
#+----+----+
#|time|data|
#+----+----+
#|0023|   g|
#|0025|   h|
#|0026|   x|
#|0031|   y|
#|0034|   z|
#+----+----+


from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().orderBy("time")
df.withColumn("lag1", F.lag("time").over(w)).withColumn("lag2", F.lag("data").over(w))\
  .withColumn("collect", F.sort_array(F.collect_list(F.array("lag1","lag2")).over(w),False)).drop("lag1","lag2")\
  .show(truncate=False)


#+----+----+-----+-----+-----+-----+
#|time|data|time1|data1|time2|data2|
#+----+----+-----+-----+-----+-----+
#|0023|g   |null |null |null |null |
#|0025|h   |0023 |g    |null |null |
#|0026|x   |0025 |h    |0023 |g    |
#|0031|y   |0026 |x    |0025 |h    |
#|0034|z   |0031 |y    |0026 |x    |
#+----+----+-----+-----+-----+-----+

+----+----+
time|data|
+----+----+
0023| g|
0025| h|
0026| x|
0031| y|
0034| z|
+----+----+

+----+----+-------------------------------------------------+
time|data|collect |
+----+----+-------------------------------------------------+
0023|g |[[,]] |
0025|h |[[0023, g], [,]] |
0026|x |[[0025, h], [0023, g], [,]] |
0031|y |[[0026, x], [0025, h], [0023, g], [,]] |
0034|z |[[0031, y], [0026, x], [0025, h], [0023, g], [,]]|
+----+----+-------------------------------------------------+

In [22]:
df.show() #sample dataframe
#+----+----+
#|time|data|
#+----+----+
#|0023|   g|
#|0025|   h|
#|0026|   x|
#|0031|   y|
#|0034|   z|
#+----+----+


from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().orderBy("time").rowsBetween(-2,Window.currentRow)


df.withColumn("collect", F.sort_array(F.collect_list(F.array("time","data")).over(w),False))\
  .show(truncate=False)


#+----+----+-----+-----+-----+-----+
#|time|data|time1|data1|time2|data2|
#+----+----+-----+-----+-----+-----+
#|0023|g   |null |null |null |null |
#|0025|h   |0023 |g    |null |null |
#|0026|x   |0025 |h    |0023 |g    |
#|0031|y   |0026 |x    |0025 |h    |
#|0034|z   |0031 |y    |0026 |x    |
#+----+----+-----+-----+-----+-----+


+----+----+
time|data|
+----+----+
0023| g|
0025| h|
0026| x|
0031| y|
0034| z|
+----+----+

+----+----+---------------------------------+
time|data|collect |
+----+----+---------------------------------+
0023|g |[[0023, g]] |
0025|h |[[0025, h], [0023, g]] |
0026|x |[[0026, x], [0025, h], [0023, g]]|
0031|y |[[0031, y], [0026, x], [0025, h]]|
0034|z |[[0034, z], [0031, y], [0026, x]]|
+----+----+---------------------------------+

In [23]:
+----+----+-------------------------------------------------+
|time|data|collect                                          |
+----+----+-------------------------------------------------+
|0023|g   |[[,]]                                            |
|0025|h   |[[0023, g], [,]]                                 |
|0026|x   |[[0025, h], [0023, g], [,]]                      |
|0031|y   |[[0026, x], [0025, h], [0023, g], [,]]           |
|0034|z   |[[0031, y], [0026, x], [0025, h], [0023, g], [,]]|
+----+----+-------------------------------------------------+

In [24]:
time | data | time | data | time | data
0023 |    g | null | null | null | null
0025 |    h | 0023 |    g | null | null
0026 |    x | 0025 |    h | 0023 |    g
0031 |    y | 0026 |    x | 0025 |    h 
0034 |    z | 0031 |    y | 0026 |    x 

In [25]:
list=[['["A","B"]']]

df=spark.createDataFrame(list,['col'])

df.show()

+---------+
 col|
+---------+
["A","B"]|
+---------+

In [26]:
from pyspark.sql import functions as F
df.withColumn("new_col", F.split(F.regexp_replace("col", '\[|]| |"', ''),",")).printSchema()

root
-- col: string (nullable = true)
-- new_col: array (nullable = true)
 |-- element: string (containsNull = true)

In [27]:
list=[[ 10 ,        10 , '11/10/20' , 'Water Polo'],
      [ 20 ,        20 , '11/22/19' , 'Cricket']]

df1=spark.createDataFrame(list,['Latitude','Longitude','Date','Event'])

df1.show()


list1=[[ 20 ,        20 ,'11/10/20' ,      90],
       [ 12 ,        12 , '11/10/20' ,      80],
       [10 ,       10 , '11/22/19' ,      34],
       [18 ,        18 , '11/22/19' ,      45]]

df2=spark.createDataFrame(list1,['Latitude','Longitude','Date','Weather'])

df2.show()


+--------+---------+--------+----------+
Latitude|Longitude| Date| Event|
+--------+---------+--------+----------+
 10| 10|11/10/20|Water Polo|
 20| 20|11/22/19| Cricket|
+--------+---------+--------+----------+

+--------+---------+--------+-------+
Latitude|Longitude| Date|Weather|
+--------+---------+--------+-------+
 20| 20|11/10/20| 90|
 12| 12|11/10/20| 80|
 10| 10|11/22/19| 34|
 18| 18|11/22/19| 45|
+--------+---------+--------+-------+

In [28]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("Date")

df1.join(df2.withColumnRenamed("Latitude","Latitude1")\
            .withColumnRenamed("Longitude","Longitude1"),['Date'])\
   .withColumn("Distance",F.sqrt(F.pow(F.col("Latitude")-F.col("Latitude1"),2)+\
                                 F.pow(F.col("Longitude")-F.col("Longitude1"),2)))\
   .withColumn("min_Distance", F.min("Distance").over(w))\
   .filter('Distance=min_Distance')\
   .select("Latitude","Longitude","Date","Weather","Event").show()

+--------+---------+--------+-------+----------+
Latitude|Longitude| Date|Weather| Event|
+--------+---------+--------+-------+----------+
 10| 10|11/10/20| 80|Water Polo|
 20| 20|11/22/19| 45| Cricket|
+--------+---------+--------+-------+----------+

In [29]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("Date")

cond=F.min(F.sqrt(F.pow(F.col("Latitude")-F.col("Latitude1"),F.lit(2))+\
                                 F.pow(F.col("Longitude")-F.col("Longitude1"),F.lit(2)))).over(w)==\
         F.sqrt(F.pow(F.col("Latitude")-F.col("Latitude1"),F.lit(2))+\
                                 F.pow(F.col("Longitude")-F.col("Longitude1"),F.lit(2)))
             

df1.join(df2.withColumnRenamed("Latitude","Latitude1")\
            .withColumnRenamed("Longitude","Longitude1")\
            .withColumnRenamed("Date","Date1"),cond).show()


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2542866440325154> in <module> 
 12 df1.join(df2.withColumnRenamed("Latitude","Latitude1")\
 13 . withColumnRenamed ( "Longitude" , "Longitude1" ) \ 
 ---> 14 .withColumnRenamed("Date","Date1"),cond).show()
 
 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 382 """
 383 if isinstance ( truncate , bool ) and truncate : 
 --> 384 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 385 else : 
 386 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o3041.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 264.0 failed 1 times, most recent failure: Lost task 0.0 in stage 264.0 (TID 3970, localhost, executor driver): java.lang.UnsupportedOperationException: Cannot evaluate expression: min(SQRT((POWER(cast((input[0, bigint, true] - input[4, bigint, true]) as double), 2.0) + POWER(cast((input[1, bigint, true] - input[5, bigint, true]) as double), 2.0)))) windowspecdefinition(input[2, string, true], specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$()))
	at org.apache.spark.sql.catalyst.expressions.Unevaluable$class.doGenCode(Expression.scala:321)
	at org.apache.spark.sql.catalyst.expressions.WindowExpression.doGenCode(windowExpressions.scala:285)
	at org.apache.spark.sql.catalyst.expressions.Expression.genCodeInternal(Expression.scala:144)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$$anonfun$genCodeWithFallback$1$$anonfun$apply$1.apply(EdgeExpressionCodegen.scala:246)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$$anonfun$genCodeWithFallback$1$$anonfun$apply$1.apply(EdgeExpressionCodegen.scala:246)
	at scala.Option.getOrElse(Option.scala:121)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$$anonfun$genCodeWithFallback$1.apply(EdgeExpressionCodegen.scala:244)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$$anonfun$genCodeWithFallback$1.apply(EdgeExpressionCodegen.scala:244)
	at scala.Option.getOrElse(Option.scala:121)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.genCodeWithFallback(EdgeExpressionCodegen.scala:242)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.streamlineIRCodeGen(EdgeExpressionCodegen.scala:293)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.streamlineCodeGen(EdgeExpressionCodegen.scala:285)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.doGenCode(EdgeExpressionCodegen.scala:1195)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.genCode(EdgeExpressionCodegen.scala:179)
	at org.apache.spark.sql.catalyst.expressions.Expression.genCode(Expression.scala:118)
	at org.apache.spark.sql.catalyst.expressions.codegen.GeneratePredicate$.create(GeneratePredicate.scala:49)
	at org.apache.spark.sql.catalyst.expressions.codegen.GeneratePredicate$.create(GeneratePredicate.scala:40)
	at org.apache.spark.sql.catalyst.expressi

In [31]:
list=[[7005410,               544,            1, 0.0026476993,'aa'],
      [7005410,               549,             2,-2.6975607E-4,'aa'],
      [7005410,               626,             3, 2.0409889E-4,'bb'],
      [7005410,               840,             2, 3.6301462E-5,'cc'],
      [7005410,              1192,             3, 2.2148499E-5,'dd']]

df=spark.createDataFrame(list,['partner','productId','mediumtype','prediction','yo'])

df.show()


+-------+---------+----------+-------------+---+
partner|productId|mediumtype| prediction| yo|
+-------+---------+----------+-------------+---+
7005410| 544| 1| 0.0026476993| aa|
7005410| 549| 2|-2.6975607E-4| aa|
7005410| 626| 3| 2.0409889E-4| bb|
7005410| 840| 2| 3.6301462E-5| cc|
7005410| 1192| 3| 2.2148499E-5| dd|
+-------+---------+----------+-------------+---+

In [32]:
df.filter('yo="aa"').show()

+-------+---------+----------+-------------+---+
partner|productId|mediumtype| prediction| yo|
+-------+---------+----------+-------------+---+
7005410| 544| 1| 0.0026476993| aa|
7005410| 549| 2|-2.6975607E-4| aa|
+-------+---------+----------+-------------+---+

In [33]:
df.filter("yo='aa'").show()

+-------+---------+----------+-------------+---+
partner|productId|mediumtype| prediction| yo|
+-------+---------+----------+-------------+---+
7005410| 544| 1| 0.0026476993| aa|
7005410| 549| 2|-2.6975607E-4| aa|
+-------+---------+----------+-------------+---+

In [34]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("partner").orderBy(F.col("prediction").asc())

df.withColumn("rowNum", F.row_number().over(w))\
  .filter('rowNum<=5').drop("rowNum").show()

+-------+---------+----------+-------------+
partner|productId|mediumtype| prediction|
+-------+---------+----------+-------------+
7005410| 549| 2|-2.6975607E-4|
7005410| 1192| 3| 2.2148499E-5|
7005410| 840| 2| 3.6301462E-5|
7005410| 626| 3| 2.0409889E-4|
7005410| 544| 1| 0.0026476993|
+-------+---------+----------+-------------+

In [35]:
df.filter('(row_number() over (partition by partner order by productId)) <=5').show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o348.filter.
: org.apache.spark.sql.AnalysisException: It is not allowed to use window functions inside WHERE and HAVING clauses;
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$class.failAnalysis(CheckAnalysis.scala:46)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.failAnalysis(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ExtractWindowExpressions$$anonfun$apply$28.applyOrElse(Analyzer.scala:2124)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ExtractWindowExpressions$$anonfun$apply$28.applyOrElse(Analyzer.scala:2121)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1$$anonfun$2.apply(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:107)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsDown$1.apply(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsDown(AnalysisHelper.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ExtractWindowExpressions$.apply(Analyzer.scala:2121)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ExtractWindowExpressions$.apply(Analyzer.scala:1962)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:112)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1$$anonfun$apply$1.apply(RuleExecutor.scala:109)
	at scala.collection.LinearSeqOptimized$class.foldLeft(LinearSeqOptimized.scala:124)
	at scala.collection.immutable.List.foldLeft(List.scala:84)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:109)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$execute$1.apply(RuleExecutor.scala:101)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:101)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$executeSameContext(Analyzer.scala:137)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:131)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor$$anonfun$executeAndTrack$1.apply(RuleExecutor.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:88)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:79)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:115)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:114)
	at org.apache.spark.sql.